In [171]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import datetime as dt

In [161]:
news = pd.read_csv("../data/News/news.csv")
stock = pd.read_csv("../data/Stock/stock.csv", index_col=0)

In [162]:
merge = pd.DataFrame(columns=["symbol", "date", "content", "preprice", "latprice", "match", "sentiment", "score"])

In [163]:
for title in ["symbol", "date", "content", "match", "sentiment"]:
    merge[title] = news[title]
merge["score"] = news["match"] * news["sentiment"]

In [164]:
idx = 0
new = news.iloc[idx]
symbol = new["symbol"].lower()
symbol = symbol if not symbol == "googl" else "goog"
date = new["date"]
if date in stock.index:
    merge.iloc[idx, merge.columns.get_loc('preprice')] = stock.loc[date][symbol]
    print(merge.iloc[idx]["preprice"], stock.loc[date][symbol])


In [182]:
for idx, new in tqdm(news.iterrows()):
    symbol = new["symbol"].lower()
    symbol = symbol if not symbol == "googl" else "goog"
    date = new["date"]
    if date in stock.index:
        # print(stock.loc[date][symbol])
        merge.iloc[idx, merge.columns.get_loc('preprice')] = stock.loc[date][symbol]
    next_day = str((dt.datetime.strptime(date, "%Y-%m-%d") + dt.timedelta(days=1)).date())
    if next_day in stock.index:
        merge.iloc[idx, merge.columns.get_loc('latprice')] = stock.loc[next_day][symbol]

25278it [00:06, 3763.55it/s]


In [183]:
merge.to_csv("../data/merge.csv")